In [24]:
import torch
import torchvision as tv
from PIL import Image, ImageDraw

In [31]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(1, 8, 5), #60
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2), #30
    torch.nn.Flatten(),
    torch.nn.Linear(8 * 30 * 30, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 10)
)
variables = model.parameters()

In [32]:
batch = 1000
optimizer = torch.optim.Adam(variables)

for epoch in range(1000):
    optimizer.zero_grad()
    
    DATA0 = torch.empty(batch, 1, 64, 64)
    TARGET0 = torch.empty(batch, dtype = torch.int64)
    for sample in range(batch):
        count = torch.randint(10, ()).item()
        POSITIONS = torch.randint(64, (count, 2))
        image = Image.new('L', (64, 64))
        draw = ImageDraw.Draw(image)
        for index in range(count):
            draw.ellipse([tuple(POSITIONS[index] - 4), tuple(POSITIONS[index] + 4)], fill = 255)
        DATA0[sample] = tv.transforms.functional.to_tensor(image)
        TARGET0[sample] = count
    
    
    ACTIVATION0 = model(DATA0)
    LOSS0 = torch.nn.functional.cross_entropy(ACTIVATION0, TARGET0)
    VALUE0 = ACTIVATION0.argmax(1)
    ACCURACY0 = torch.eq(VALUE0, TARGET0).float().mean()
    LOSS0.backward()
    optimizer.step()
    
    print("%4d %12.3f %4.3f" % \
          (epoch, LOSS0, ACCURACY0), flush = True)

   0        2.304 0.119
   1        2.282 0.109
   2        2.241 0.083
   3        2.192 0.092
   4        2.152 0.080
   5        2.121 0.076
   6        2.072 0.115
   7        2.045 0.117
   8        1.979 0.138
   9        1.939 0.160


In [33]:
CONFUSION0 = torch.zeros(10,10)
for sample in range(batch):
    CONFUSION0[TARGET0[sample], VALUE0[sample]] += 1
print(CONFUSION0)

tensor([[ 0.,  0.,  0., 98.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  3., 96.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  2., 71., 23.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., 27., 35.,  0., 24.,  0.,  5.,  0.],
        [ 0.,  0.,  0., 12., 17.,  0., 38.,  0., 37., 10.],
        [ 0.,  0.,  0.,  1.,  6.,  0., 13.,  0., 54., 22.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  8.,  0., 70., 26.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  2.,  0., 69., 29.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., 64., 38.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 55., 42.]])
